In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Metabolic_Rate"
cohort = "GSE41168"

# Input paths
in_trait_dir = "../../input/GEO/Metabolic_Rate"
in_cohort_dir = "../../input/GEO/Metabolic_Rate/GSE41168"

# Output paths
out_data_file = "../../output/preprocess/Metabolic_Rate/GSE41168.csv"
out_gene_data_file = "../../output/preprocess/Metabolic_Rate/gene_data/GSE41168.csv"
out_clinical_data_file = "../../output/preprocess/Metabolic_Rate/clinical_data/GSE41168.csv"
json_path = "../../output/preprocess/Metabolic_Rate/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE41168_family.soft.gz', 'GSE41168_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Metabolic_Rate/GSE41168/GSE41168_family.soft.gz
Matrix file: ../../input/GEO/Metabolic_Rate/GSE41168/GSE41168_series_matrix.txt.gz


Background Information:
!Series_title	"Resveratrol supplementation does not improve metabolic function in non-obese women with normal glucose tolerance"
!Series_summary	"Resveratrol has been reported to improve metabolic function in metabolically-abnormal rodents and humans, but has not been studied in non-obese people with normal glucose tolerance.  We conducted a randomized, double-blind, placebo-controlled trial to evaluate the metabolic effects of 12 weeks of resveratrol supplementation (75 mg/day) in non-obese, postmenopausal women with normal glucose tolerance.  Although resveratrol supplementation was well-tolerated and increased plasma resveratrol concentration without adverse effects, it did not change body composition, resting metabolic rate, plasma lipids, or inflammatory markers.  A two-stage hyperinsulinemic-euglycemic clamp procedure, in conjunction with stable isotopically-labeled tracer infusions, demonstrated that resveratrol did not increase liver, skeletal muscle, or

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Dict, Any, Callable, Optional, List, Union

# 1. Evaluate gene expression data availability
# Based on the Series_summary and Series_overall_design, this dataset contains gene expression data from
# subcutaneous abdominal adipose tissue and skeletal muscle samples. It's not just miRNA or methylation data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Metabolic Rate):
# From the background information, this is a study on metabolic function and resveratrol supplementation
# The "sample group" in row 1 indicates different interventions that could affect metabolic rate
trait_row = 1  # sample group: calorie restrictive/placebo/resveratrol

# For age:
# No explicit age information in the sample characteristics dictionary
age_row = None

# For gender:
# Gender information is in row 3, but it's constant (all Female)
# Since it's constant, it's not useful for our association study
gender_row = None  # Although it exists in row 3, all samples are female

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> Union[float, None]:
    """Convert trait values to binary numeric values."""
    if value is None:
        return None
    
    # Extract value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert sample group to binary based on intervention
    # Treatment groups (resveratrol, calorie restriction) are 1, control (placebo) is 0
    if "resveratrol" in value.lower() or "calorie restrictive" in value.lower():
        return 1.0
    elif "placebo" in value.lower():
        return 0.0
    else:
        return None

def convert_age(value: str) -> Union[float, None]:
    """Convert age values to numeric values."""
    # Not used since age_row is None, but defined for compatibility
    return None

def convert_gender(value: str) -> Union[float, None]:
    """Convert gender values to binary numeric values."""
    # Not used since gender_row is None, but defined for compatibility
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    try:
        # Load the clinical data from the matrix file
        # The format of GEO matrix files requires special parsing
        clinical_data = pd.read_table(f"{in_cohort_dir}/GSE41168_series_matrix.txt.gz", 
                                    compression='gzip', comment='!', 
                                    header=0)
        
        # Extract clinical features using the function from the library
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical data
        print("Clinical Data Preview:")
        print(preview_df(selected_clinical_df))
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical data to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to: {out_clinical_data_file}")
        
    except Exception as e:
        print(f"Error during data extraction: {e}")
        # If we can't extract valid trait data, set trait availability to False
        is_trait_available = False
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available
        )
        print("Updated cohort info due to extraction failure.")


Error during data extraction: argument of type 'float' is not iterable
Updated cohort info due to extraction failure.


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 54613
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers from the output
# The identifiers like '1007_s_at', '1053_at', etc. are typical Affymetrix probe IDs,
# not standard human gene symbols (which would look like BRCA1, TP53, etc.)
# These probe IDs need to be mapped to standard gene symbols for meaningful analysis

# Based on biomedical knowledge, these are Affymetrix probe IDs that require mapping
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which keys in the gene annotation dictionary correspond to identifiers and gene symbols
# From the preview, 'ID' column matches the gene identifiers from gene expression data
# 'Gene Symbol' column contains the corresponding gene symbols
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Extract the mapping between probes and gene symbols from gene annotation
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

print("Mapping dataframe preview:")
print(preview_df(mapping_df))

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

print("\nGene expression data preview:")
print(f"Number of genes: {len(gene_data)}")
print(f"Sample of gene symbols: {list(gene_data.index[:10])}")
print(f"First 5 rows and 5 columns:\n{gene_data.iloc[:5, :5]}")

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save gene expression data to CSV
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to: {out_gene_data_file}")


Mapping dataframe preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}



Gene expression data preview:
Number of genes: 21278
Sample of gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']
First 5 rows and 5 columns:
          GSM1009750  GSM1009751  GSM1009752  GSM1009753  GSM1009754
Gene                                                                
A1BG        0.441593    0.352073    1.117518    0.955207    0.434547
A1BG-AS1    1.048902    0.755683    0.995292    1.054459    1.099495
A1CF        0.678257    0.384777    1.069259    0.453898    0.494588
A2M         7.708893    7.795638    9.462428    9.861125    7.292437
A2M-AS1     2.338703    2.537749    3.986187    3.813036    2.212537


Gene expression data saved to: ../../output/preprocess/Metabolic_Rate/gene_data/GSE41168.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Re-examine the clinical data from the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print out the sample characteristics to verify available rows
characteristics_dict = get_unique_values_by_row(clinical_data)
print("Sample characteristics dictionary:")
print(characteristics_dict)

# Based on the background information, this is a study about metabolic function
# but the specific metabolic rate values may not be explicitly stored in the characteristics
# We'll use the sample group (row 1) as our trait, as it indicates different interventions
# that would affect metabolic rate (calorie restriction, resveratrol, placebo)

# Define conversion functions for the clinical features
def convert_treatment_to_trait(value):
    if not isinstance(value, str):
        return None
    
    # Extract value after the colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    else:
        value = value.lower()
    
    # Treatment groups (resveratrol, calorie restriction) are 1, control (placebo) is 0
    if "resveratrol" in value or "calorie restrictive" in value:
        return 1.0
    elif "placebo" in value:
        return 0.0
    else:
        return None

def convert_gender(value):
    if not isinstance(value, str):
        return None
    
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    else:
        value = value.lower()
    
    if 'female' in value:
        return 0.0
    elif 'male' in value:
        return 1.0
    else:
        return None

# Create the clinical dataframe using the correct row indices based on sample characteristics
try:
    clinical_df = geo_select_clinical_features(
        clinical_data,
        trait="Metabolic_Rate",
        trait_row=1,  # Row 1 contains sample group (intervention type)
        convert_trait=convert_treatment_to_trait,
        gender_row=3 if 3 in characteristics_dict and 'gender' in str(characteristics_dict[3]).lower() else None,  # Row 3 for gender if it exists
        convert_gender=convert_gender,
        age_row=None  # No age information available
    )
    
    print("Clinical data preview:")
    print(preview_df(clinical_df.T))  # Transpose for better viewing
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.T.to_csv(out_clinical_data_file)
    print(f"Saved clinical data to {out_clinical_data_file}")
    
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data_normalized)
    print(f"Shape of linked data: {linked_data.shape}")
    
    # 4. Handle missing values in the linked data
    linked_data_cleaned = handle_missing_values(linked_data, 'Metabolic_Rate')
    print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")
    
    # 5. Check if the trait and demographic features are biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, 'Metabolic_Rate')
    
    # 6. Validate the dataset and save cohort information
    note = "Dataset contains gene expression data from muscle and adipose tissue samples, comparing placebo, resveratrol supplementation, and calorie restriction interventions."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note=note
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Saved processed linked data to {out_data_file}")
    else:
        print("Dataset validation failed. Final linked data not saved.")
        
except Exception as e:
    print(f"Error in processing clinical data: {e}")
    # If we failed to extract clinical data, update the cohort info
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=None,
        df=pd.DataFrame(),
        note="Failed to extract clinical data. Gene expression data is available but missing trait information."
    )
    print("Dataset validation failed due to missing clinical data. Only gene data saved.")

Shape of gene data after normalization: (19845, 140)


Saved normalized gene data to ../../output/preprocess/Metabolic_Rate/gene_data/GSE41168.csv


Sample characteristics dictionary:
{0: ['tissue: muscle', 'tissue: adipose tissue'], 1: ['sample group: calorie restrictive', 'sample group: placebo', 'sample group: resveratrol'], 2: ['treatment: before', 'treatment: after'], 3: ['gender: Female']}
Clinical data preview:
{'Metabolic_Rate': [1.0, 1.0, 0.0, 0.0, 0.0], 'Gender': [0.0, 0.0, 0.0, 0.0, 0.0]}
Saved clinical data to ../../output/preprocess/Metabolic_Rate/clinical_data/GSE41168.csv
Shape of linked data: (140, 19847)


Shape of linked data after handling missing values: (140, 19847)
For the feature 'Metabolic_Rate', the least common label is '0.0' with 46 occurrences. This represents 32.86% of the dataset.
The distribution of the feature 'Metabolic_Rate' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 140 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Saved processed linked data to ../../output/preprocess/Metabolic_Rate/GSE41168.csv
